Generating plots that explore iridoplasts using the LOPC model. This notebook is for generating publication-ready plots, which will be automatically exported to the LaTeX directory. To follow the process of exploring the data, look at the other notebooks.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
import pandas as pd
# import xyzpy as xyz
# from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
from pathlib import Path
import sys
from tqdm import tqdm
# plotting
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
import colorcet
import panel as pn
import panel.widgets as pnw
from bokeh.io import export_png, export_svg
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from scipy.signal import find_peaks
from bokeh.models import PrintfTickFormatter

In [ ]:
hv.extension("bokeh", inline=False, case_sensitive_completion=True)  # use matplotlib because rendering bokeh to svg is broken
pn.config.throttled = True  # don't update interactive plots until mouse is unclicked

# default_color_cycle = hv.Cycle("Colorblind")  # Ruth doesn't like the inclusion of yellow, which is fair enough
default_color_cycle = hv.Cycle(colorcet.glasbey_dark)
default_dash_cycle = hv.Cycle(["solid", "dashed", "dashdot", "dotted", "dotdash"])
universal_opts = dict(fontscale=2, title="")
matplotlib_opts = dict(fig_inches=5, aspect=2, fig_latex=True)
bokeh_opts = dict(width=700, height=300)
opts.defaults(opts.Curve(**universal_opts|bokeh_opts, color=default_color_cycle, line_width=1.5),
              opts.Scatter(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Image(**universal_opts|bokeh_opts),
              opts.Slope(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Area(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Overlay(**universal_opts|bokeh_opts),
              opts.Layout(**universal_opts|bokeh_opts),
              opts.GridSpace(**universal_opts|bokeh_opts),
              )

xarray_engine='h5netcdf'

In [ ]:
root = Path.cwd().parent.parent.parent  # depth of parents depends on if this is running in JupyterLab or Notebook

In [ ]:
code_path = root / r"research"

In [ ]:
data_path = code_path / r"notebooks/data"

In [ ]:
archive_path = root / r"thesis/LaTeX/chapters/chapter_3"

In [ ]:
fig_path = archive_path / "fig_chapter_3"

In [ ]:
if not code_path in sys.path:
    sys.path.append(str(code_path))
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    # combo_length,
    # estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    visualise_multilayer,
    complex_elements,
    indexer_from_dataset,
    fix_bin_labels,
    mean_and_std,
    assign_high_from_mean_and_std,
    assign_low_from_mean_and_std,
    assign_high_and_low,
    max_min_pos,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

EDIT: This does not work but I'm leaving this here so a future researcher can avoid the rabbithole I fell down.

In [ ]:
# # This is the idiomatic way to record all generated figures with holoviews
# # This does NOT work in JupyterLab: see https://github.com/holoviz/holoviews/issues/3570
# # This also does not work in Jupyter Notebook
# # It's just utterly broken

# hv.archive.auto(root=str(archive_path), export_name="fig_chapter_2") 

# Load/define datasets

## Basic datasets

In [ ]:
# chunks for plotting or integrating over angle
undisordered_run_number = 2
pds_chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 16,
    "passive_layer_thickness": 32,
    "theta": 16,
    "num_periods": 1,
}

periodic_ds = xr.open_mfdataset(
    data_path / f"run_{undisordered_run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=pds_chunks,
)
periodic_ds = assign_derived_attrs(
    periodic_ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

# Prepare for Plots

## Pre-processing

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
RTA_cycle = hv.Cycle([blue, yellow, red])

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

In [ ]:
unpolarised_RTA = ["R", "T", "A"]
s_polarised_RTA = ["Rs", "Ts", "As"]
p_polarised_RTA = ["Rp", "Tp", "Ap"]
reflectance = ["Rs", "Rp", "R"]
transmittance = ["Ts", "Tp", "T"]
absorptance = ["As", "Ap", "A"]
per_oscillator_RTA = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]

In [ ]:
pre_process_for_plots = partial(pre_process_for_plots, strict=False)

In [ ]:
# WARNING: all these datasets will be fundamentally changed after this cell, to the extent that it can't be run twice
# For consistency, keep important calculations in the preceding section!
periodic_ds = pre_process_for_plots(periodic_ds)  # the delta=0 case
# singles_pds = pre_process_for_plots(singles_pds)  # periodic_ds indexed by singles_ds, below

# uncorrelated_aggregated_ds = pre_process_for_plots(uncorrelated_aggregated_ds) # uncorrelated delta<0.15, higher precision in elt and period
# correlated_aggregated_ds = pre_process_for_plots(correlated_aggregated_ds) # correlated delta<0.15, higher precision in elt and period
# aggregated_ds = pre_process_for_plots(aggregated_ds) # combination of above

# singles_ds_01_04 = pre_process_for_plots(singles_ds_01_04)  # individual run data delta<0.4, coarse
# singles_ds_05 = pre_process_for_plots(singles_ds_05)  # individual run data delta=0.5, coarse
# # singles_ds_10 = pre_process_for_plots(singles_ds_10)  # individual run data delta=1, coarse, inefficient - never use this!
# correlated_ds_10 = pre_process_for_plots(correlated_ds_10)  # much faster version of above without unnecessary reindexing
# singles_ds = pre_process_for_plots(singles_ds)  # merge of the above - convenient but inefficient due to ragged structure
# stats_ds = pre_process_for_plots(stats_ds)  # calculated mean and standard deviation of individual run data

# # + restacked versions of all the above
# ref = pre_process_for_plots(ref)
# crs_1 = pre_process_for_plots(crs_1)
# # norm_1 = pre_process_for_plots(norm_1)
# # restacked_norm_1 = pre_process_for_plots(restacked_norm_1)
# # diff_1 = pre_process_for_plots(diff_1)
# # restacked_diff_1 = pre_process_for_plots(restacked_diff_1)

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    restacked_pds = restack_plt_to_period(periodic_ds)  # the delta=0 case
#     restacked_spds = restack_plt_to_period(singles_pds)  # periodic_ds indexed by singles_ds, below

#     restacked_uads = restack_plt_to_period(uncorrelated_aggregated_ds) # uncorrelated delta<0.15, higher precision in elt and period
#     restacked_cads = restack_plt_to_period(correlated_aggregated_ds) # correlated delta<0.15, higher precision in elt and period
#     restacked_ads = restack_plt_to_period(aggregated_ds)

#     restacked_sds_01_04 = restack_plt_to_period(singles_ds_01_04)  # individual run data delta<0.4, coarse
#     restacked_sds_05 = restack_plt_to_period(singles_ds_05)  # individual run data delta=0.5, coarse
#     # restacked_sds_10 = restack_plt_to_period(singles_ds_10)  # individual run data delta=1, coarse, inefficient - never use this!
#     restacked_cds_10 = restack_plt_to_period(correlated_ds_10)  # much faster version of above without unnecessary reindexing
#     restacked_sds = restack_plt_to_period(singles_ds)  # merge of the above - convenient but inefficient due to ragged structure
#     restacked_stats_ds = restack_plt_to_period(stats_ds)  # calculated mean and standard deviation of individual run data

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")
delta_dim = hv.Dimension("delta", label="Δ")
elt_dim = hv.Dimension("excitonic_layer_thickness", label="d_e", unit="nm")

## Plotting functions

### Useful lines

#### Function to do linewidth maths

In [ ]:
def linewidth_calculator_factory(centre, linewidth):
    return partial(linewidth_calculator, centre=centre, linewidth=linewidth)

#### Lorentz lines

I want some sort of metric for 'near the resonance' and 'far from the resonance'. The natural unit of distance in this instance is the linewidth. The linewidth is given in rad/s so there need to be some conversions to get the equivalent lines in the plots by wavelength, but they are roughly symmetrical around the peak wavelength.

Based on the plots of the refractive index below, I think I will consider 'near' to be 'within two linewidths', and 'far' to be 'at least four linewidths away'.

In [ ]:
# resonance_line = hv.VLine(680, label='LO resonance wavelength').opts(line_dash='dotted')

# Convert from rad/s to Hz
lorentz_linewidth_frequency = default_oscillator_params["lorentz_linewidth"] / (2*np.pi)

In [ ]:
lorentz_line_frequency = linewidth_calculator_factory(convert_wavelength_and_frequency(680e-9), lorentz_linewidth_frequency)

def lorentz_line_wavelength(x=None):
    x = -x if x is not None else x
    return convert_wavelength_and_frequency(lorentz_line_frequency(x))

In [ ]:
def lorentz_vlines(x=0, scale=1, mode='wavelength', **kwargs):
    if mode == 'wavelength':
        line_func = lorentz_line_wavelength
    elif mode == 'frequency':
        line_func = lorentz_line_frequency
    else:
        raise TypeError(f"mode should be 'wavelength' or 'frequency', not {mode}")
        
    match x:
        case [*xs]:
            line_pos = [line_func(x)/scale for x in xs]
        case x:
            line_pos = line_func(x)/scale
            
    return vlines(line_pos, **kwargs)

### Useful functions

#### Select a wavelength or wavelength range based on the distance from the resonance in linewidths.

In [ ]:
def select_lorentz_line(da, lorentz_line=0, window_radius=0):
    if window_radius == 0:
        wavelength = lorentz_line_wavelength(lorentz_line) * 1e9
        wavelength_sel_method = "nearest"
    else:
        wavelength = slice(
            lorentz_line_wavelength(lorentz_line - window_radius) * 1e9,
            lorentz_line_wavelength(lorentz_line + window_radius) * 1e9,
        )
        wavelength_sel_method = None
    da = da.sel(wavelength=wavelength, method=wavelength_sel_method)
    
    return da

In [ ]:
def cross_section(
    dataset,
    variable=None,
    wavelength=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1,
):
    """Select and integrate"""
    da = dataset[variable] if variable is not None else dataset
    integration_dims = []

    if not integrate_angle:
        if theta is not None:
            da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        integration_dims.append("theta")

    match wavelength:
        case None:
            da = select_lorentz_line(
                da, lorentz_line=lorentz_line, window_radius=window_radius
            )

            if window_radius != 0:
                integration_dims.append("wavelength")
        case (start, stop, *_):
            da = da.sel(wavelength=slice(*wavelength))
            integration_dims.append("wavelength")
        case _:
            da = sel_slice_or_nearest(da, "wavelength", wavelength)

    if integration_dims:  # if the list isn't empty
        da = integrate_da(
            da, integration_dims, weighting=1, normalisation=normalisation
        )

    return da

### Useful functions

#### Plot a comparison of the reflectance and absorptance of the LOPC with that of the reference slab.

In [ ]:
opts_R = [opts.Curve(color=blue, ylim=(0,1)), opts.Image(cmap='viridis', clim=(0,1)), opts.QuadMesh(cmap='viridis', clim=(0,1)), opts.Violin(violin_fill_color=blue)]

def plot_R(variable="R", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot

# # test
# plot_R(dataset=restacked_pds.mean("run", keep_attrs=True).sel(period=200, excitonic_layer_thickness=50, num_periods=10).squeeze(), x="wavelength", y="theta").opts(cmap="cividis", clim=(None, None))

In [ ]:
opts_T = [opts.Curve(color=yellow, ylim=(0,1)), opts.Image(cmap='cividis', clim=(0,1)), opts.QuadMesh(cmap='cividis', clim=(0,1)), opts.Violin(violin_fill_color=yellow)]

def plot_T(variable="T", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_T)
    return plot

In [ ]:
opts_A = [opts.Curve(color=red, ylim=(0,1)), opts.Image(cmap='inferno', clim=(0,1)), opts.QuadMesh(cmap='inferno', clim=(0,1)), opts.Violin(violin_fill_color=red)]

def plot_A(variable="A", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_A)
    return plot

In [ ]:
def plot_vars_to_funcs(plot_vars):
    var_func_mapping = {
        "R": plot_R,
        "T": plot_T,
        "A": plot_A,
        "Rs": partial(plot_R, variable="Rs"),
        "Ts": partial(plot_T, variable="Ts"),
        "As": partial(plot_A, variable="As"),
        "Rp": partial(plot_R, variable="Rp"),
        "Tp": partial(plot_T, variable="Tp"),
        "Ap": partial(plot_A, variable="Ap"),
        "R_per_oscillator": partial(plot_R, variable="R_per_oscillator"),
        "T_per_oscillator": partial(plot_T, variable="T_per_oscillator"),
        "A_per_oscillator": partial(plot_A, variable="A_per_oscillator"),
        "Rs_per_oscillator": partial(plot_R, variable="Rs_per_oscillator"),
        "Ts_per_oscillator": partial(plot_T, variable="Ts_per_oscillator"),
        "As_per_oscillator": partial(plot_A, variable="As_per_oscillator"),
        "Rp_per_oscillator": partial(plot_R, variable="Rp_per_oscillator"),
        "Tp_per_oscillator": partial(plot_T, variable="Tp_per_oscillator"),
        "Ap_per_oscillator": partial(plot_A, variable="Ap_per_oscillator"),
    }

    plot_funcs = []
    for var in plot_vars:
        try:
            func = var_func_mapping[var]
        except KeyError:
            func = partial(plot_var, variable=var)
        plot_funcs.append(func)

    return plot_funcs

In [ ]:
# new version
def plot_RTA(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta,
    title="",
    include=["LOPC", "CRS_1"],
    plot_vars=["R", "T", "A"],
    label_override=None,
    label_append=None,
):
    label_field = None  # for debugging
    label_append = "" if label_append is None else label_append

    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plot_funcs = [
        partial(func, x="wavelength", label_field=label_field)
        for func in plot_vars_to_funcs(plot_vars)
    ]
    curves = []
    if "LOPC" in include:
        lopc_label = " (LOPC)" if label_override is None else label_override
        lopc_label += label_append
        lopc_sel = restacked_pds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)
        lopc_curves = [
            plot_func(dataset=lopc_sel, label_append=lopc_label).opts(line_dash="solid")
            for plot_func in plot_funcs
        ]
        curves += lopc_curves
    if "CRS_1" in include:
        crs_1_label = " (CRS)" if label_override is None else label_override
        crs_1_label += label_append
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = sel_or_integrate(crs_1_sel, "theta", theta, normalisation=1)
        crs_1_curves = [
            plot_func(dataset=crs_1_sel, label_append=crs_1_label).opts(
                line_dash="dashed"
            )
            for plot_func in plot_funcs
        ]
        curves += crs_1_curves
    if "FRS_1" in include:
        frs_1_label = " (FRS)" if label_override is None else label_override
        frs_1_label += label_append
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = sel_or_integrate(frs_1_sel, "theta", theta, normalisation=1)
        frs_1_curves = [
            plot_func(dataset=frs_1_sel, label_append=frs_1_label).opts(
                line_dash="dotted"
            )
            for plot_func in plot_funcs
        ]
        curves += frs_1_curves

    overlay = hv.Overlay(curves).opts(
        opts.Curve(
            ylim=(0, 1),
            ylabel="Intensity",
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, theta=theta)}",
        ),
    )

    return overlay


# # test
# display(
#     plot_RTA(200, 50, 20, 0, "test\n", include=["LOPC", "CRS_1"]).opts(
#         legend_position="right"
#     )
# )

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         (10, 50),
#         "test RA only\n",
#         include=["LOPC", "CRS_1"],
#         plot_vars=["R", "A"],
#     ).opts(opts.Overlay(legend_position="right"))
# )

# display(
#     plot_RTA(
#         200,
#         50,
#         20,
#         75,
#         "test\n",
#         include=["LOPC"],
#         plot_vars=["R_per_oscillator", "A_per_oscillator"],
#         label_append=" test",
#         label_override="OVERRIDDEN",
#     ).opts(opts.Curve(ylim=(None, None)), opts.Overlay(legend_position="right"))
# )

#### Plot a comparison of normal incidence to integrated

In [ ]:
def plot_comparison(*comparison_params: tuple[dict, list["opts"]], plot_func=plot_RTA, **shared_params):
    param_opts = [(shared_params|comp_params, comp_opts) for comp_params, comp_opts in comparison_params]
    plots = [plot_func(**comp_params).opts(*comp_opts) for comp_params, comp_opts in param_opts]
    return plots

In [ ]:
def compare_RTA(*args, opts_cycle=None, plot_func=plot_RTA, **shared_params):
    default_opts = [
        [opts.Curve(line_dash=style)]
        for style in ["solid", "dashed", "dotted", "dotdash", "dashdot"]
    ]
    opts_cycle = default_opts if opts_cycle is None else opts_cycle

    # comparison_params = list(zip(args, opts_cycle))

    plots = plot_comparison(
        *zip(args, opts_cycle), plot_func=plot_func, **shared_params
    )
    overlay = hv.Overlay(plots).opts(opts.Overlay(legend_position="right"))

    return overlay


# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
#     "include": ["LOPC"],
# }
# compare_RTA({"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, **shared_params)

In [ ]:
compare_RTA_normal_vs_integrated = partial(
    compare_RTA,
    {"theta": (0, 75), "label_override": " (integrated)"},
    {"theta": 0, "label_override": " (θ = 0)"},
    include=["LOPC"],
)

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
# }
# compare_RTA_normal_vs_integrated(**shared_params)

#### Plot an enhancement factor.

In [ ]:
def plot_ef(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} enhancement factor")
        plot *= hv.HLine(1).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} enhancement factor",
            cmap="RdBu_r",
            clim=(0.5, 1.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot


# # test
# sel_1 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 30}
# sel_2 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 0}
# sel_3 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10}
# display(
#     (
#         plot_ef("As", restacked_norm_1, sel_1, "nearest", "test\n")
#         + plot_ef("As", restacked_norm_2, sel_2, title="test2\n")
#     ).cols(1)
# )
# display(
#     plot_ef("As", restacked_norm_1, sel_3, title="test3\n", x="theta", y="wavelength").opts(clim=(0, 2), cmap="RdBu_r")
# )

#### Plot a difference factor.

In [ ]:
def plot_df(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} difference factor")
        plot *= hv.HLine(0).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} difference factor",
            cmap="RdBu_r",
            clim=(-0.5, 0.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot

# Plots

## Light spectra from Endler

In [ ]:
Endler_path = data_path / "Endler_data"

In [ ]:
forest_shade_data = pd.read_csv(Endler_path / "forest_shade_sunny_Endler_1993.csv", names=["Wavelength", "Intensity"])
woodland_shade_data = pd.read_csv(Endler_path / "woodland_shade_sunny_Endler_1993.csv", names=["Wavelength", "Intensity"])
small_gap_data = pd.read_csv(Endler_path / "direct_sunlight_small_gaps_Endler_1993.csv", names=["Wavelength", "Intensity"])
no_canopy_data = pd.read_csv(Endler_path / "direct_sunlight_no_canopy_Endler_1993.csv", names=["Wavelength", "Intensity"])
pisonia_data = pd.read_csv(Endler_path / "forest_shade_pisonia_Endler_1993.csv", names=["Wavelength", "Intensity"])
leaf_data = pd.read_csv(Endler_path / "leaf_sun_Endler_1993.csv", names=["Wavelength", "Intensity"])

In [ ]:
Endler_data = {
    "Forest Shade": forest_shade_data,
    "Woodland Shade": woodland_shade_data,
    "Small Gap": small_gap_data,
    "No Canopy": no_canopy_data,
    "Pisonia": pisonia_data,
    "Leaf": leaf_data,
}

According to the caption of Fig. 6, photon intensity was meant to be recorded with 2 nm spacing. I will assume that this is correct and every measurement was meant to be made exactly on the wavelengths 400, 402, ... 700 nm. The deviations from this are ascribed to error in WebPlotDigitizer.

The curves from Endler have offsets applied to them so they can be presented side-by-side in the same plots. Reverse the offsets:

In [ ]:
Endler_data_offsets = {
    "Forest Shade": 0.050,
    "Woodland Shade": 0,
    "Small Gap": 0.050,
    "No Canopy": 0.115,
    "Pisonia": 0,
    "Leaf": 0,
}

In [ ]:
for label in Endler_data:
    df = Endler_data[label]
    offset = Endler_data_offsets[label]
    df["Wavelength"] = np.arange(400, 701, 2)  # align DataFrames
    df["Intensity"] -= offset  # remove offsets
    # df["Normalised Intensity"] = df["Intensity"]/df["Intensity"].max()  # normalise to max values
    df["Normalised Intensity"] = df["Intensity"]/df.loc[lambda df: df["Wavelength"]==550]["Intensity"].iloc[0]  # normalise to 550 nm values

In [ ]:
overlay = hv.Overlay(
    [
        # hv.Curve((df["Wavelength"], df["Normalised Intensity"]), kdims=[wavelength_dim], vdims=["Normalised Intensity"], label=label)
        df.hvplot(x="Wavelength", y="Normalised Intensity", label=label)
        for label, df in Endler_data.items()# if label != "Pisonia"
    ]
)

In [ ]:
overlay = overlay.redim(Wavelength=wavelength_dim)

overlay.opts(
    opts.Curve(
        ylim=(0, 1.2),
        line_dash=hv.Cycle(
            ["solid", "dashdot", "dashed", "dotted", "dotdash", "solid"]
        ),
        color=hv.Cycle(
            [
                default_color_cycle.values[0],
                default_color_cycle.values[1],
                default_color_cycle.values[4],
                default_color_cycle.values[3],
                default_color_cycle.values[2],
                default_color_cycle.values[5],
            ]
        ),
    ),
    opts.Overlay(
        legend_position="bottom_right",
        legend_cols=3,
        legend_opts={"background_fill_alpha": 0, "border_line_alpha": 0},
        fontscale=1.2,  # this plot will be 4x bigger in paper, so don't need fontscale=2
    ),
)

In [ ]:
hv.save(overlay, filename=fig_path / "light_spectra_550_Endler_1993", fmt="png", toolbar=False)

In [ ]:
from scipy import integrate

In [ ]:
for df in Endler_data.values():
    df["Normalised by Total"] = df["Intensity"]/(integrate.trapezoid(df["Intensity"], x=df["Wavelength"]))  # normalise to total intensity
max_intensity = max([max(df["Normalised by Total"]) for label, df in Endler_data.items() if label != "Leaf"])
for df in Endler_data.values():
    df["Normalised Intensity"] = df["Normalised by Total"]/max_intensity  # normalise to max intensity in whole dataset

In [ ]:
overlay = hv.Overlay(
    [
        # hv.Curve((df["Wavelength"], df["Normalised Intensity"]), kdims=[wavelength_dim], vdims=["Normalised Intensity"], label=label)
        df.hvplot(x="Wavelength", y="Normalised Intensity", label=label)
        for label, df in Endler_data.items() if label != "Leaf"
    ]
)

In [ ]:
overlay = overlay.redim(Wavelength=wavelength_dim)

overlay.opts(
    opts.Curve(
        ylim=(0, 1),
        line_dash=hv.Cycle(
            ["solid", "dashdot", "dashed", "dotted", "dotdash", "solid"]
        ),
        color=hv.Cycle(
            [
                default_color_cycle.values[0],
                default_color_cycle.values[1],
                default_color_cycle.values[4],
                default_color_cycle.values[3],
                default_color_cycle.values[2],
                default_color_cycle.values[5],
            ]
        ),
    ),
    opts.Overlay(
        legend_position="bottom_right",
        legend_cols=3,
        legend_opts={"background_fill_alpha": 0, "border_line_alpha": 0},
        fontscale=1.2,  # this plot will be 4x bigger in paper, so don't need fontscale=2
    ),
)

In [ ]:
hv.save(overlay, filename=fig_path / "light_spectra_total_max_Endler_1993", fmt="png", toolbar=False)

In [ ]:
overlay = hv.Overlay(
    [
        # hv.Curve((df["Wavelength"], df["Normalised Intensity"]), kdims=[wavelength_dim], vdims=["Normalised Intensity"], label=label)
        df.hvplot(x="Wavelength", y="Intensity", label=label)
        for label, df in Endler_data.items() if label != "Leaf"
    ]
)

In [ ]:
overlay = overlay.redim(Wavelength=wavelength_dim)

overlay.opts(
    opts.Curve(
        ylim=(0, 0.123),
        line_dash=hv.Cycle(
            ["solid", "dashdot", "dashed", "dotted", "dotdash", "solid"]
        ),
        color=hv.Cycle(
            [
                default_color_cycle.values[0],
                default_color_cycle.values[1],
                default_color_cycle.values[4],
                default_color_cycle.values[3],
                default_color_cycle.values[2],
                default_color_cycle.values[5],
            ]
        ),
    ),
    opts.Overlay(
        legend_position="bottom_right",
        legend_cols=3,
        legend_opts={"background_fill_alpha": 0, "border_line_alpha": 0},
        fontscale=1.2,  # this plot will be 4x bigger in paper, so don't need fontscale=2
    ),
)

In [ ]:
hv.save(overlay, filename=fig_path / "light_spectra_total_Endler_1993", fmt="png", toolbar=False)

## Direct-diffuse differentiation

In [ ]:
P = 300
elt = 30
N = 10
theta = (0, 45)
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(
    {"theta": 10, "label_override": " (θ = 10)"},
    {"theta": 20, "label_override": " (θ = 20)"},
    {"theta": 30, "label_override": " (θ = 30)"},
    period=P,
    excitonic_layer_thickness=elt,
    num_periods=N,
    plot_vars=["R"],
    include=["LOPC"],
)
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
P = 200
elt = 30
N = 10
theta = (0, 45)
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(
    {"theta": 10, "label_override": " (θ = 10)"},
    {"theta": 20, "label_override": " (θ = 20)"},
    {"theta": 30, "label_override": " (θ = 30)"},
    period=P,
    excitonic_layer_thickness=elt,
    num_periods=N,
    plot_vars=["R"],
    include=["LOPC"],
)
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
restacked_pds

In [ ]:
plot_RTA(period=200, excitonic_layer_thickness=20, num_periods=10, theta=0, include=["LOPC"])

### Broadband reflectance (generous assumptions)

In [ ]:
N = 30  # actual optimum is 17
elt = 40
theta_range = (0, 70)
period_range = (elt, elt+310)
normal_dataset = integrate_da(
    restacked_pds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    restacked_pds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
overlay = (
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="R_tot", ylim=(0, None), xlim=period_range),
    opts.Overlay(legend_position="top_left"),
)

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_comparison_broadband_generous", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_df(variable="R", dataset=(normal_dataset - angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (difference)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_difference_broadband_generous", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_ratio_broadband_generous", fmt="png", toolbar=False)

In [ ]:
inverse_normal_dataset = 1 - normal_dataset
inverse_angle_dataset = 1 - angle_dataset
overlay = (
    compare_RTA(
        {"dataset": inverse_angle_dataset, "label": "1-R (integrated)"},
        {"dataset": inverse_normal_dataset, "label": "1-R (θ = 0)"},
        plot_func=plot_R,
        # label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="1-R_tot", ylim=(None, 1), xlim=period_range),
    opts.Overlay(legend_position="bottom_left"),
)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(inverse_normal_dataset / inverse_angle_dataset), x="period")
).redim(period=period_dim, R="1-R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "inverse_Rtot_normal_vs_integrated_ratio_broadband_generous", fmt="png", toolbar=False)

In [ ]:
# peak is at Lambda = 180
P = 180
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylabel="R", ylim=(0, None)),opts.Overlay(legend_position="top_right"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P180_elt40_N30_th_normal_vs_integrated_comparison_broadband_generous", fmt="png", toolbar=False)

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None), title=""))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P180_elt40_N30_broadband_generous", fmt="png", toolbar=False)

In [ ]:
# second peak is at Lambda = 260
P = 260
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylabel="R", ylim=(0, None)),opts.Overlay(legend_position="top_left"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P260_elt40_N30_th_normal_vs_integrated_comparison_broadband_generous", fmt="png", toolbar=False)

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None), title=""))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P260_elt40_N30_broadband_generous", fmt="png", toolbar=False)

### Narrowband reflectance (generous assumptions)

In [ ]:
lorentz_line = 2
window_radius = 1
N = 30  # actual optimum is 17
elt = 40
theta_range = (0, 70)
period_range = (elt, elt+310)

normal_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
lines = lorentz_vlines(
    [lorentz_line - window_radius, lorentz_line + window_radius], scale=1e-9
).opts(
    opts.VLine(line_color="white", line_dash="dotted"),
)
shade = hv.VSpan(lines.VLine.I.x, lines.VLine.II.x).opts(alpha=0.1, color="gray")

overlay = (
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="R_tot", ylim=(0, None), xlim=period_range),
    opts.Overlay(legend_position="top_left"),
)

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_comparison_narrowband_generous", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_df(variable="R", dataset=(normal_dataset - angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (difference)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_difference_narrowband_generous", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_ratio_narrowband_generous", fmt="png", toolbar=False)

In [ ]:
inverse_normal_dataset = 1 - normal_dataset
inverse_angle_dataset = 1 - angle_dataset
overlay = (
    compare_RTA(
        {"dataset": inverse_angle_dataset, "label": "1-R (integrated)"},
        {"dataset": inverse_normal_dataset, "label": "1-R (θ = 0)"},
        plot_func=plot_R,
        # label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="1-R_tot", ylim=(None, 1), xlim=period_range),
    opts.Overlay(legend_position="bottom_left"),
)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(inverse_normal_dataset / inverse_angle_dataset), x="period")
).redim(period=period_dim, R="1-R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "inverse_Rtot_normal_vs_integrated_ratio_narrowband_generous", fmt="png", toolbar=False)

In [ ]:
# peak is at Lambda = 260
P = 260
fig = (
    compare_RTA_normal_vs_integrated(
        period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"]
    )
    * lines
    * shade
)
fig.opts(
    opts.Curve(ylabel="R", ylim=(0, None)),
    opts.Overlay(legend_position="top_left"),
    opts.VLine(line_color=green),
)

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P260_elt40_N30_th_normal_vs_integrated_comparison_narrowband_generous", fmt="png", toolbar=False)

In [ ]:
fig = (
    plot_R(
        dataset=restacked_pds.sel(
            period=P, excitonic_layer_thickness=elt, num_periods=N
        )
        .sel(theta=slice(*theta_range))
        .squeeze(),
        x="wavelength",
        y="theta",
    )
    * lines
)
fig.opts(opts.Image(clim=(0, None)), opts.VLine(line_color="white"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P260_elt40_N30_narrowband_generous", fmt="png", toolbar=False)

### Broadband reflectance (conservative assumptions)

In [ ]:
N = 10  # actual optimum is 17
elt = 20
theta_range = (0, 45)
period_range = (elt, elt+310)
normal_dataset = integrate_da(
    restacked_pds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    restacked_pds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
overlay = (
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="R_tot", ylim=(0, None), xlim=period_range),
    opts.Overlay(legend_position="top_left"),
)

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_comparison_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_df(variable="R", dataset=(normal_dataset - angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (difference)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_difference_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_ratio_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
inverse_normal_dataset = 1 - normal_dataset
inverse_angle_dataset = 1 - angle_dataset
overlay = (
    compare_RTA(
        {"dataset": inverse_angle_dataset, "label": "1-R (integrated)"},
        {"dataset": inverse_normal_dataset, "label": "1-R (θ = 0)"},
        plot_func=plot_R,
        # label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="1-R_tot", ylim=(None, 1), xlim=period_range),
    opts.Overlay(legend_position="bottom_left"),
)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(inverse_normal_dataset / inverse_angle_dataset), x="period")
).redim(period=period_dim, R="1-R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "inverse_Rtot_normal_vs_integrated_ratio_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
# peak is at Lambda = 180
P = 180
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylabel="R", ylim=(0, None)),opts.Overlay(legend_position="top_right"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P180_elt40_N30_th_normal_vs_integrated_comparison_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None), title=""))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P180_elt40_N30_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
# second peak is at Lambda = 260
P = 260
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylabel="R", ylim=(0, None)),opts.Overlay(legend_position="top_left"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P260_elt40_N30_th_normal_vs_integrated_comparison_broadband_conservative", fmt="png", toolbar=False)

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None), title=""))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P260_elt40_N30_broadband_conservative", fmt="png", toolbar=False)

### Narrowband reflectance (conservative assumptions)

In [ ]:
lorentz_line = 2
window_radius = 1
N = 10  # actual optimum is 17
elt = 30
theta_range = (0, 70)
period_range = (elt, elt+310)

normal_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
lines = lorentz_vlines(
    [lorentz_line - window_radius, lorentz_line + window_radius], scale=1e-9
).opts(
    opts.VLine(line_color="white", line_dash="dotted"),
)
shade = hv.VSpan(lines.VLine.I.x, lines.VLine.II.x).opts(alpha=0.1, color="gray")

overlay = (
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="R_tot", ylim=(0, None), xlim=period_range),
    opts.Overlay(legend_position="top_left"),
)

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_comparison_narrowband_conservative", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_df(variable="R", dataset=(normal_dataset - angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (difference)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_difference_narrowband_conservative", fmt="png", toolbar=False)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")
).redim(period=period_dim, R="R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "Rtot_normal_vs_integrated_ratio_narrowband_conservative", fmt="png", toolbar=False)

In [ ]:
inverse_normal_dataset = 1 - normal_dataset
inverse_angle_dataset = 1 - angle_dataset
overlay = (
    compare_RTA(
        {"dataset": inverse_angle_dataset, "label": "1-R (integrated)"},
        {"dataset": inverse_normal_dataset, "label": "1-R (θ = 0)"},
        plot_func=plot_R,
        # label_field=None,
        x="period",
    )
).redim(period=period_dim)

overlay.opts(
    opts.Curve(ylabel="1-R_tot", ylim=(None, 1), xlim=period_range),
    opts.Overlay(legend_position="bottom_left"),
)

In [ ]:
overlay = (
    plot_ef(variable="R", dataset=(inverse_normal_dataset / inverse_angle_dataset), x="period")
).redim(period=period_dim, R="1-R_tot (ratio)")
overlay.opts(opts.Curve(xlim=period_range, color=blue))

In [ ]:
hv.save(overlay, filename=fig_path / "inverse_Rtot_normal_vs_integrated_ratio_narrowband_conservative", fmt="png", toolbar=False)

In [ ]:
# peak is at Lambda = 260
P = 260
fig = (
    compare_RTA_normal_vs_integrated(
        period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"]
    )
    * lines
    * shade
)
fig.opts(
    opts.Curve(ylabel="R", ylim=(0, None)),
    opts.Overlay(legend_position="top_left"),
    opts.VLine(line_color=green),
)

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_1D_wl_P260_elt40_N30_th_normal_vs_integrated_comparison_narrowband_conservative", fmt="png", toolbar=False)

In [ ]:
fig = (
    plot_R(
        dataset=restacked_pds.sel(
            period=P, excitonic_layer_thickness=elt, num_periods=N
        )
        .sel(theta=slice(*theta_range))
        .squeeze(),
        x="wavelength",
        y="theta",
    )
    * lines
)
fig.opts(opts.Image(clim=(0, None)), opts.VLine(line_color="white"))

In [ ]:
hv.save(fig, filename=fig_path / "R_LOPC_2D_wl_th_P260_elt40_N30_narrowband_conservative", fmt="png", toolbar=False)

### Placeholder

In [ ]:
lorentz_line = -11
window_radius = 2
N = 10
elt = 30
theta_range = (0, 45)
normal_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    select_lorentz_line(restacked_pds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
lines = lorentz_vlines(
    [lorentz_line - window_radius, lorentz_line + window_radius], scale=1e-9
).opts(
    opts.VLine(line_color=green, line_dash="dotted"),
)

(
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim).opts(opts.Curve(ylim=(None, None), xlim=(100, 340), title="Integrated by wavelength"))

In [ ]:
(plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")).redim(
    period=period_dim, R="R (normal / integrated)"
).opts(opts.Curve(xlim=(100, 340)))

In [ ]:
# peak is at Lambda = 180 
P = 180
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])*lines
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)*lines
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
(
    plot_R(dataset=(normal_dataset - angle_dataset), label_field=None, x="period")
    * hv.HLine(0)
).redim(period=period_dim, R="R (normal - integrated)",
).opts(
    opts.Curve(ylim=(None, None), title=""),
    opts.HLine(line_dash="dotted"),
)

In [ ]:
# peak is at Lambda = 190
P = 190
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])*lines
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_pds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)*lines
fig.opts(opts.Image(clim=(0,None)))

## placholder